In [11]:
import os
import pandas as pd
from google.cloud import storage
import time
from io import StringIO

import gspread 
from oauth2client.service_account import ServiceAccountCredentials

In [19]:
# google cloud sheets
relative_path = '../../deep-flash-sa.json'
file_path = os.path.abspath(relative_path)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = file_path
storage_client = storage.Client()

# google sheets 
spreadsheet_key = "1KZJmFezoZlc9drr-y5grwWE3u9UMqMSzAafzq89Ky7w"
scope = "https://spreadsheets.google.com/feeds"
credential = "./ads-sheets-notebook-47e00ba6aad5.json"

In [ ]:
#feature pipeline
def read_data_from_gcs(bucket_name, folder, filename, delimiter=','):
    start_time = time.time()  # Start measuring time
    storage_client = storage.Client()
    blob = storage_client.get_bucket(bucket_name).blob(f'{folder}/{filename}')
    csv_data = blob.download_as_text()
    df = pd.read_csv(StringIO(csv_data), delimiter=delimiter)
    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Read {filename} complete. Elapsed time: {elapsed_time:.2f} seconds")
    return df

def get_data_webvisitor():
    data = read_data_from_gcs('perqara-dendrobium', 'website-visitor/processed', 'website-visitor-synthetic.csv')
    return data

def get_data_ads(sheet=None):
    start_time = time.time()  # Start measuring time
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credential, scope)
    worksheet = gspread.authorize(credentials).open_by_key(spreadsheet_key).worksheet(sheet)
    data = worksheet.get_all_values()
    headers = data.pop(0)
    elapsed_time = time.time() - start_time  # Calculate elapsed time
    print(f"Read {sheet} sheet complete. Elapsed time: {elapsed_time:.2f} seconds")
    return pd.DataFrame(data, columns=headers)
    
def filter_date(df, start, end):
    df = df[(df['date'] >= start) & (df['date'] <= end)].copy()
    return df

def convert_date_to_datetime(df):
    df['date'] = pd.to_datetime(df['date'])
    return df

def filter_column(df):
    df = df[['date','cost_google', 'cost_meta']].copy()
    return df 
 
def convert_columns_to_int(df, columns):
    for col in columns:
        df[col] = df[col].astype(int)
    return df

def create_lagged_features(data, lag):
    lagged_data = data.copy()
    for i in range(1, lag + 1):
        lagged_data[f'lag_{i}'] = data['count'].shift(i)
    data = lagged_data.fillna(lagged_data.mean(numeric_only=True))
    return data
        
def merge_dataframes(df1, df2):
    merged_df = pd.merge(df1, df2, on='date', how='outer')
    merged_df = merged_df.sort_values('date')
    return merged_df

df_website_visitor = get_data_webvisitor()
df_website_ads = get_data_ads(sheet='ads')

# transform website
df_web_visitor = filter_date(df_website_visitor, '2023-07-01', '2023-12-10')
df_web_visitor = convert_date_to_datetime(df_web_visitor)
df_web_visitor = create_lagged_features(df_web_visitor, 7)

# transform ads
df_ads = filter_column(df_website_ads)
df_ads = convert_date_to_datetime(df_ads)
df_ads = convert_columns_to_int(df_ads, ['cost_google', 'cost_meta'])

# merge data
merged_df = merge_dataframes(df_web_visitor, df_ads)
merged_df.to_csv("feature.csv", index=False)

In [ ]:
#training pipeline

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_absolute_error
import mlflow

def load_data(file_path):
    # Load data
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    return df

def prepare_data(df):
    # Prepare features and target
    X = df.drop(['count'], axis=1)
    y = df['count']
    return X, y

def hyperparameter_tuning(X, y, cv_splits=5):
    # Time Series Split for cross-validation
    tscv = TimeSeriesSplit(n_splits=cv_splits)

    # Hyperparameter tuning with cross-validation using MAE
    param_grid = {
        'learning_rate': [0.01, 0.1, 0.2, 0.3],
        'max_depth': [3, 4, 5, 6, 7, 8, 9],
        'min_child_weight': [1, 2, 3, 4],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'n_estimators': [100, 200, 300, 400, 500]
    }

    xgb = XGBRegressor(objective='reg:squarederror')

    # Define MAE as the scoring metric
    scoring_metric = make_scorer(mean_absolute_error, greater_is_better=False)

    # RandomizedSearchCV with TimeSeriesSplit
    random_search = RandomizedSearchCV(xgb, param_distributions=param_grid, scoring=scoring_metric, cv=tscv)
    random_search.fit(X, y)

    # Get the best hyperparameters
    best_params = random_search.best_params_
    print(f'Best Hyperparameters: {best_params}')

    return best_params

def train_and_evaluate(X, y, best_params, cv_splits=5):
    # Time Series Split for cross-validation
    tscv = TimeSeriesSplit(n_splits=cv_splits)

    # Train XGBoost model with the best hyperparameters
    best_xgb_model = XGBRegressor(objective='reg:squarederror', **best_params)

    # Evaluate the model using cross-validation scores with MAE
    cv_scores = cross_val_score(best_xgb_model, X, y, cv=tscv, scoring='neg_mean_absolute_error')
    mean_cv_score = np.mean(cv_scores)
    
    best_xgb_model.fit(X, y)
    
    # Display or log the results
    print(f'Cross-Validation Scores (MAE): {cv_scores}')
    print(f'Mean Cross-Validation Score (MAE): {mean_cv_score}')

    return best_xgb_model, cv_scores, mean_cv_score

def xgboost_pipeline(file_path):
    # Step 1: Load Data
    df = load_data(file_path)

    # Step 2: Prepare Data
    X, y = prepare_data(df)

    # Step 3: Hyperparameter Tuning
    best_hyperparams = hyperparameter_tuning(X, y)

    # Step 4: Train and Evaluate
    best_model, cv_scores, mean_cv_score = train_and_evaluate(X, y, best_hyperparams)

    return best_model

# Example usage:
best_trained_model = xgboost_pipeline("feature.csv")

mlflow.xgboost.save_model(best_trained_model, 'best_xgb_model')

In [ ]:
#inference pipeline

def load_model(model_name):
    return mlflow.xgboost.load_model(model_name)

def convert_date_to_datetime(df):
    df['date'] = pd.to_datetime(df['date'])
    return df

def create_forecast_frame(feature_data, forecast_horizon):
    start_date = pd.to_datetime(feature_data.index[-1]) + pd.DateOffset(days=1)
    date_range = pd.date_range(start_date, periods=forecast_horizon, freq='D')
    forecast_df = pd.DataFrame(index=date_range, columns=["forecasted_value"])
    return forecast_df

def forecast(model, feature_data, forecast_temp, forecast_horizon):
    forecast_array = []
    columns_to_append = ['cost_google', 'cost_meta']

    latest_data_row = feature_data.iloc[-1, 0:7].to_numpy().reshape(1, -1)

    for i in range(forecast_horizon):
        latest_data_row = np.append(latest_data_row, forecast_temp[columns_to_append].iloc[i])
        latest_data_row = latest_data_row.reshape(1, -1)
        predicted_result = model.predict(latest_data_row)

        new_data_row = np.concatenate((predicted_result, latest_data_row[0]))

        forecast_array.append(new_data_row)

        latest_data_row = new_data_row[0:7].reshape(1, -1)

    forecast_value = [array[0] for array in forecast_array]
    forecast_temp["forecasted_value"] = forecast_value
    
    return forecast_temp

# load model
loaded_model = load_model("best_xgb_model") #path

# load data
df = load_data("feature.csv")

# load variable ads google and meta projection
df_ads_proj = get_data_ads(sheet='ads_projection')

# transform ads
df_ads_proj = convert_date_to_datetime(df_ads_proj)
df_ads_proj.set_index('date', inplace=True)

forecast_df = create_forecast_frame(df, 21)

forecast_template = pd.merge(forecast_df, df_ads_proj, left_index=True, right_index=True)

result = forecast(loaded_model, df, forecast_template, 21)

In [262]:
result

,forecasted_value,cost_google,cost_meta
2023-12-11,253.436340,100000,250000
2023-12-12,254.577499,100000,250000
2023-12-13,807.095581,1050000,800000
2023-12-14,781.113220,1050000,800000
2023-12-15,734.999573,1050000,800000
2023-12-16,653.861816,1050000,800000
2023-12-17,649.127319,1050000,800000
2023-12-18,268.523163,100000,100000
2023-12-19,233.098618,100000,100000
2023-12-20,954.652344,560000,600000
